## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## Camera calibration using chessboard images helper function

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

# Input - none - hard-coded values of camera calibartion picked up
# Return - camera matrix (mtx) and distortion coefficients (dist)
def calibrate_camera():
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            #img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
            #cv2.imshow('img',img)
            #cv2.waitKey(500)

    #cv2.destroyAllWindows()

    #Camera calibration, given object points, image points, and the shape of the grayscale image:

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    
    return mtx, dist




## Now, we'll use the calibration to undistort any given image - test run on specific
## Only used for debug, not required for actual production run


In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# Call the calibrate function and get our mtx and dist 
mtx, dist = calibrate_camera()

img = mpimg.imread('test_images/test1.jpg')  # Random calibration image - anything can be chosen
undist = cv2.undistort(img, mtx, dist, None, mtx)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()

ax1.imshow(img)
ax1.set_title('Original Image', fontsize=40)

ax2.imshow(undist)
ax2.set_title('Undistored image', fontsize=40)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)


cv2.destroyAllWindows()

## Threshold binary image helper function

In [ ]:
import matplotlib.image as mpimg

# Helper function to return threshold binary image
# Input - undistorted image, threshold values optional
# Output - Combined binary image
def threshold_binary(img, s_thresh=(100, 200), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HLS color space and separate the H channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
    
    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1

    return combined_binary

# Call the calibrate function and get our mtx and dist 
#mtx, dist = calibrate_camera()

# Debug values below. Uncomment as needed for production
#img = mpimg.imread('test_images/test3.jpg')
#undist = cv2.undistort(img, mtx, dist, None, mtx)
#cbinary = threshold_binary(undist)
#f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
#f.tight_layout()

#ax1.imshow(img)
#ax1.set_title('Original Image', fontsize=40)

#ax2.imshow(cbinary, cmap='gray')
#ax2.set_title('Pipeline Result', fontsize=40)
#plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

## Perspective Transform to get a bird's eyeview of the road

In [ ]:
import matplotlib.image as mpimg

# Helper function to get a bird's eyeview of the road of a given image
# Input - image
# Output - warped image, transformative matrix (M) and Inverse transformative matrix (Minv)
def p_transform(img):
    # Source coords for perspective transform
    src = np.float32([[240,719],
                        [579,450],
                        [712,450],
                        [1165,719]])
    # Dest coords for perspective transform
    dst = np.float32([[300,719],
                        [300,0],
                        [900,0],
                        [900,719]])

    # Call the calibrate function and get our mtx and dist 
    mtx, dist = calibrate_camera()


    
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    cbinary = threshold_binary(undist)
    
    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    # Warp the image using OpenCV warpPerspective()
    Minv = cv2.getPerspectiveTransform(dst, src)


    warped = cv2.warpPerspective(cbinary, M, (cbinary.shape[1], cbinary.shape[0]))
    
    return M, Minv, warped


# Modified function of p_transform to put in parallel lines for project rubric purposes
# No real use in actual pipeline implementation
# Input - image
# Output - warped image, transformative matrix (M) and Inverse transformative matrix (Minv)
def p_transform_lines(img):
    # Source coords for perspective transform
    src = np.float32([[240,719],
                        [579,450],
                        [712,450],
                        [1165,719]])
    # Dest coords for perspective transform
    dst = np.float32([[300,719],
                        [300,0],
                        [900,0],
                        [900,719]])

    # Call the calibrate function and get our mtx and dist 
    mtx, dist = calibrate_camera()


    
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    cbinary = threshold_binary(undist)
    
    pts = np.array([src[0],src[3],src[2],src[1]], np.int32)
    pts = pts.reshape((-1,1,2))
    copy = img.copy()
    cv2.polylines(copy,[pts],True,(255,0,0), thickness=3)

    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    # Warp the image using OpenCV warpPerspective()
    Minv = cv2.getPerspectiveTransform(dst, src)


    warped = cv2.warpPerspective(cbinary, M, (cbinary.shape[1], cbinary.shape[0]))
    
        
    pts_dst = np.array([dst[0],dst[3],dst[2],dst[1]], np.int32)
    pts_dst = pts_dst.reshape((-1,1,2))
    warped_copy = warped.copy()
    cv2.polylines(warped_copy,[pts_dst],True,(255,0,0), thickness=3)
    
    return M, Minv, warped_copy, copy

# Debug lines below. Commented for production run
# Uncomment for debugging
#img = mpimg.imread('test_images/test3.jpg')

#M, Minv, warped_copy, copy = p_transform_lines(img)

#f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
#f.tight_layout()

#ax1.imshow(copy)
#ax1.set_title('Original Image', fontsize=40)

#ax2.imshow(warped_copy, cmap='gray')
#ax2.set_title('Pipeline Result', fontsize=40)
#plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)



## Detect lane pixels and fit to find the lane boundary.

In [35]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Helper function to detect lane pixels and fit the lane boundary
# Input - image
# Ouptut - binary_warped, left_fit, right_fit, lefty, righty, ploty
def fit_lane(img):
    
    # Call the perspective transform helper function
    M, Minv, binary_warped = p_transform(img)
    
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]/2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
    # Comment below lines for production
    #plt.imshow(out_img)
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    #plt.xlim(0, 1280)
    #plt.ylim(720, 0)
    
    #Lane line distance from center calculation
    perfect_center = img.shape[1]/2
    right_lane_pixel = rightx_base + leftx_base
    left_lane_pixel = leftx_base
    lane_x = right_lane_pixel - left_lane_pixel
    center_x = (lane_x / 2.0) + left_lane_pixel
    cms_per_pixel = 370.0 / lane_x   
    dist_from_center = (center_x - perfect_center) * cms_per_pixel
    #print(dist_from_center/100, 'm')
    
    return binary_warped, left_fit, right_fit, lefty, righty, ploty, left_fitx, right_fitx

# Comment out for production
# Uncomment for debug
#img = mpimg.imread('test_images/test1.jpg')
#fit_lane(img)

C:\Users\ram\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


0.519662921348 m


(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([  1.16276591e-04,  -3.01000278e-01,   4.55973635e+02]),
 array([  1.31948390e-04,  -1.73529481e-01,   9.24402938e+02]),
 array([640, 640, 640, ...,  79,  79,  79], dtype=int64),
 array([642, 642, 643, ...,  79,  79,  79], dtype=int64),
 array([   0.,    1.,    2.,    3.,    4.,    5.,    6.,    7.,    8.,
           9.,   10.,   11.,   12.,   13.,   14.,   15.,   16.,   17.,
          18.,   19.,   20.,   21.,   22.,   23.,   24.,   25.,   26.,
          27.,   28.,   29.,   30.,   31.,   32.,   33.,   34.,   35.,
          36.,   37.,   38.,   39.,   40.,   41.,   42.,   43.,   44.,
          45.,   46.,   47.,   48.,   49.,   50.,   51.,   52.,   53.,
          54.,   55.,   56.,   57.,   58.,   59.,   60.,   61.,   62.,
          63.,   64.,   65.,   6

## Skip the sliding windows step once you know where the lines are
Now you know where the lines are you have a fit! In the next frame of video you don't need to do a blind search again, but instead you can just search in a margin around the previous line position like this:

In [31]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Function to find lane lines given we already have a "binary_warped" image with lane lines fit
# Input binary warped image (binary_warped), left_fit, right_fit (previously detected) 
# Output -left_fit, right_fit and ploty 
def fit_lane_sliding(binary_warped, left_fit, right_fit):

    # Assume you now have a new warped binary image 
    # from the next frame of video (also called "binary_warped")
    # It's now much easier to find line pixels!
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    #plt.imshow(result)
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    #plt.xlim(0, 1280)
    #plt.ylim(720, 0)
    
    return binary_warped, left_fit, right_fit, lefty, righty, ploty, leftx, rightx, left_fitx, right_fitx

# Debug for non-production
# Comment for production

#img = mpimg.imread('test_images/test4.jpg')
#binary_warped, left_fit, right_fit, lefty, righty, ploty, left_fitx, right_fitx = fit_lane(img)

#fit_lane_sliding(binary_warped, left_fit, right_fit)


## Determine the curvature of the lane and vehicle position with respect to center.

In [32]:
# Determine the curvature of lane and vehicle position

def curvature(left_fit, right_fit, ploty, leftx, rightx):
    
    # Define y-value where we want radius of curvature
    # I'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    print(left_curverad, right_curverad)
    # Example values: 1926.74 1908.48

    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    print(left_curverad, 'm', right_curverad, 'm')
    # Example values: 632.1 m    626.2 m
    
# Debug for non-production
# Comment for production

#img = mpimg.imread('test_images/test4.jpg')
#binary_warped, left_fit, right_fit, lefty, righty, ploty, left_fitx, right_fitx = fit_lane(img)

#binary_warped, left_fit, right_fit, lefty, righty, ploty, leftx, rightx, left_fitx, right_fitx = fit_lane_sliding(binary_warped, left_fit, right_fit)
#curvature(left_fit, right_fit, ploty, leftx, rightx)

C:\Users\ram\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


4264.94960254 2256.69339317
1373.21196981 m 723.460081322 m


## Warp the detected lane boundaries back onto the original image.

In [37]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def final_image(img, warped, Minv, left_fitx, right_fitx, ploty, leftx, rightx):

    # Call the calibrate function and get our mtx and dist 
    mtx, dist = calibrate_camera()

    undist = cv2.undistort(img, mtx, dist, None, mtx)
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)

    return result

#Debug lines
img = mpimg.imread('test_images/test3.jpg')
binary_warped, left_fit, right_fit, lefty, righty, ploty, left_fitx, right_fitx = fit_lane(img)
binary_warped, left_fit, right_fit, lefty, righty, ploty, leftx, rightx, left_fitx, right_fitx = fit_lane_sliding(binary_warped, left_fit, right_fit)
M, Minv, warped = p_transform(img)
result = final_image(img, warped, Minv, left_fitx, right_fitx, ploty, leftx, rightx)
plt.imshow(result)



C:\Users\ram\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


0.653064699205 m


## Put together everything that we've created so far into a pipeline

In [38]:
# Silly hack to do camera calibration and get M and Minv values for this cell
# Source coords for perspective transform
src = np.float32([[240,719],
                    [579,450],
                    [712,450],
                    [1165,719]])
# Dest coords for perspective transform
dst = np.float32([[300,719],
                    [300,0],
                    [900,0],
                    [900,719]])

# Call the calibrate function and get our mtx and dist 
mtx, dist = calibrate_camera()

# Given src and dst points, calculate the perspective transform matrix
M = cv2.getPerspectiveTransform(src, dst)
# Warp the image using OpenCV warpPerspective()
Minv = cv2.getPerspectiveTransform(dst, src)

# Global variables
polyfit_left=None
polyfit_right=None

# Array to store previously detected lines
past_good_left_lines = []
past_good_right_lines = []

# Keep track of the mean difference between the lines so that we 
# can know if there is a deviation the lines (which indicates a bad frame)
running_mean_difference_btn_lines = 0

# Helper Function to generate a prediction for the lane lines
# Input - various co-ordinate points
# Output - Left and right lane predictions

def line_predictions(non_zeros_x, non_zeros_y, left_coordinates, right_coordinates, num_rows):

    left_x = non_zeros_x[left_coordinates]
    left_y = non_zeros_y[left_coordinates]
    
    # If no pixels were found return None
    # For left side lane
    if(left_y.size == 0 or left_x.size == 0):
        return None, None

    # Otherwise, fit the polynomial
    polyfit_left = np.polyfit(left_y, left_x, 2)

    right_x = non_zeros_x[right_coordinates]
    right_y = non_zeros_y[right_coordinates]
    
    # If no pixels were found return None
    # Repeat for right side
    if(right_y.size == 0 or right_x.size == 0):
        return None, None

    # Fit the polynomial
    polyfit_right = np.polyfit(right_y, right_x, 2)

    # If no pixels were found return None
    y_points = np.linspace(0, num_rows-1, num_rows)
    
    # Generate the lane lines from the polynomial fit
    left_x_predictions = polyfit_left[0]*y_points**2 + polyfit_left[1]*y_points + polyfit_left[2]
    right_x_predictions = polyfit_right[0]*y_points**2 + polyfit_right[1]*y_points + polyfit_right[2]
    
    # Return the predicted lane lines
    return left_x_predictions, right_x_predictions

# Brute force function to find out lane lines if we need it
# Drop everything and start with binary warped image to generate the lines freshly
# Input - Binary warped image
# Output - left and right lanes
def brute_force(warped):
    
    # Implementation is similar to fit_lane() helper function with slight modifications
    non_zeros = warped.nonzero()
    non_zeros_y = non_zeros[0]
    non_zeros_x = non_zeros[1]
    
    num_rows = warped.shape[0]
    
    histogram = np.sum(warped[warped.shape[0]/2:,:], axis=0)

    half_width = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:half_width])
    rightx_base = np.argmax(histogram[half_width:]) + half_width

    num_windows = 10
    window_height = np.int(num_rows/num_windows)
    window_half_width = 50

    min_pixels = 100

    left_coordinates = []
    right_coordinates = []

    for window in range(num_windows):
        y_max = num_rows - window*window_height
        y_min = num_rows - (window+1)* window_height

        left_x_min = leftx_base - window_half_width
        left_x_max = leftx_base + window_half_width

        good_left_window_coordinates = ((non_zeros_x >= left_x_min) & (non_zeros_x <= left_x_max) & (non_zeros_y >= y_min) & (non_zeros_y <= y_max)).nonzero()[0]
        left_coordinates.append(good_left_window_coordinates)

        if len(good_left_window_coordinates) > min_pixels:
            leftx_base = np.int(np.mean(non_zeros_x[good_left_window_coordinates]))

        right_x_min = rightx_base - window_half_width
        right_x_max = rightx_base + window_half_width

        good_right_window_coordinates = ((non_zeros_x >= right_x_min) & (non_zeros_x <= right_x_max) & (non_zeros_y >= y_min) & (non_zeros_y <= y_max)).nonzero()[0]
        right_coordinates.append(good_right_window_coordinates)

        if len(good_right_window_coordinates) > min_pixels:
            rightx_base = np.int(np.mean(non_zeros_x[good_right_window_coordinates]))

    left_coordinates = np.concatenate(left_coordinates)
    right_coordinates = np.concatenate(right_coordinates)
    
    left_x_predictions, right_x_predictions = line_predictions(non_zeros_x, non_zeros_y, left_coordinates, right_coordinates, num_rows)
    return left_x_predictions, right_x_predictions


# Function to create average lines based on previous n lines
# Input - previous lines, current lines, frames to avearge (default at 10)
# Output - same as input minus frames to average

def average_lines(previous_lines, new_line, frames = 10):

    # No new lines
    if new_line is None:
                
        # Or previous lines. Nothing to compute, so return None
        if len(previous_lines) == 0:
            return previous_lines, None
        
        # Otherwise return the previous line and last detected previous line
        else:
            return previous_lines, previous_lines[-1]
    
    # We have new lines, so let's get cracking
    else:
        # Check if there are enough previous lines than frames to average over
        if len(previous_lines) < frames:
            # No dice, just append the new line onto the existing previous line and return the values
            previous_lines.append(new_line)
            return previous_lines, new_line
        else:
            # Yes, we can average over the frames
            previous_lines[0:frames-1] = previous_lines[1:]
            previous_lines[frames-1] = new_line
            new_line = np.zeros_like(new_line)
            for i in range(frames):
                new_line += previous_lines[i]
            new_line /= frames
            
            # And return the previous lines and new lines
            return previous_lines, new_line

# Simplified version of our previous calcualte_radius_of_curvature function
# Takes in only 1 value - along the x axis
# use this 2 times - once for x and once for y unlike our original function
def simple_curvature(x_values, num_rows):
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    # If no pixels were found return None
    y_points = np.linspace(0, num_rows-1, num_rows)
    y_eval = np.max(y_points)

    # Fit new polynomials to x,y in world space
    fit_cr = np.polyfit(y_points*ym_per_pix, x_values*xm_per_pix, 2)
    curvature = ((1 + (2*fit_cr[0]*y_eval*ym_per_pix + fit_cr[1])**2)**1.5) / np.absolute(2*fit_cr[0])
    return curvature        
        
# Our final pipeline
# Input - single unprocessed raw image
# Output - image with lane line projected

# IMPORTANT NOTE - to run this function, previous cells should have been executed that defines other functions

def pipeline(img):
    # global variables to store line detected in the last frame
    global polyfit_right
    global polyfit_left
    
    # global variables to store the line coordinates in previous frames
    global past_good_right_lines
    global past_good_left_lines
    
     
    # Implementation is mostly a rehash of our previous functions with minor modifications
    
    # global variable which contains running average of the mean difference between left and right lanes
    global running_mean_difference_btn_lines
    
    # Get our image size
    img_shape = img.shape
    img_size = (img_shape[1], img_shape[0])
    
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    # get thresholded image using our previous helper function
    thresholded = threshold_binary(undist)
    
    # perform a perspective transform
    warped = cv2.warpPerspective(thresholded, M, img_size , flags=cv2.INTER_LINEAR)
    
    out_img = np.dstack((warped, warped, warped))*255
    
    non_zeros = warped.nonzero()
    non_zeros_y = non_zeros[0]
    non_zeros_x = non_zeros[1]
    
    num_rows = warped.shape[0]
    y_points = np.linspace(0, num_rows-1, num_rows)
    
    # If the previous polyfit_left or right are none, that means, we've lost the frame.
    # But no worries, scall the brute force function
    if (polyfit_left is None) or (polyfit_right is None):
        brute = True
        left_x_predictions, right_x_predictions = brute_force(warped)
    
    # Otherwise
    else:
        # Set brute to false
        brute = False
        
        # And use our code from previous functions to determine the co-ordinates
        margin = 100
        left_x_predictions = polyfit_left[0]*non_zeros_y**2 + polyfit_left[1]*non_zeros_y + polyfit_left[2]
        left_coordinates = ((non_zeros_x >= left_x_predictions - margin) & (non_zeros_x <= left_x_predictions + margin)).nonzero()[0]

        right_x_predictions = polyfit_right[0]*non_zeros_y**2 + polyfit_right[1]*non_zeros_y + polyfit_right[2]
        right_coordinates = ((non_zeros_x >= right_x_predictions - margin) & (non_zeros_x <= right_x_predictions + margin)).nonzero()[0]
        
        left_x_predictions, right_x_predictions = line_predictions(non_zeros_x, non_zeros_y, left_coordinates, right_coordinates, num_rows)
    
    if (left_x_predictions is None or right_x_predictions is None):
        if not brute:
            left_x_predictions, right_x_predictions = brute_force(warped)
    
    # Set our lines to be good by default
    is_lines_bad = False
    
    # Houston, we have a problem
    if (left_x_predictions is None or right_x_predictions is None):
        # Set that the lines are bad
        is_lines_bad = True
    else:
        mean_difference = np.mean(right_x_predictions - left_x_predictions)
        
        if running_mean_difference_btn_lines == 0:
            running_mean_difference_btn_lines = mean_difference
        
        # Here we're using a hard-coded value that the difference in lines should be
        # between 0.7 and 1.3
        # Surely there must be a better way to figure this out without hard-coding
        
        # The next few lines use various conditional statements to determine whether our lines are good or bad
        # The implementation is a bit convoluted and could be made simpler
        if (mean_difference < 0.7*running_mean_difference_btn_lines or mean_difference > 1.3*running_mean_difference_btn_lines):
            is_lines_bad = True
            if not brute:
                left_x_predictions, right_x_predictions = brute_force(warped)
                if (left_x_predictions is None or right_x_predictions is None):
                    is_lines_bad = True
                else:
                    mean_difference = np.mean(right_x_predictions - left_x_predictions)
                    if (mean_difference < 0.7*running_mean_difference_btn_lines or mean_difference > 1.3*running_mean_difference_btn_lines):
                        is_lines_bad = True
                    else:
                        is_lines_bad = False
        else:
            is_lines_bad = False
            
    if is_lines_bad:
        polyfit_left = None
        polyfit_right = None
        if len(past_good_left_lines) == 0 and len(past_good_right_lines) == 0:
            return img
        else:
            left_x_predictions = past_good_left_lines[-1]
            right_x_predictions = past_good_right_lines[-1]
    else:
        past_good_left_lines, left_x_predictions = average_lines(past_good_left_lines, left_x_predictions)
        past_good_right_lines, right_x_predictions = average_lines(past_good_right_lines, right_x_predictions)
        mean_difference = np.mean(right_x_predictions - left_x_predictions)
        running_mean_difference_btn_lines = 0.9*running_mean_difference_btn_lines + 0.1*mean_difference
    
    left_line_window = np.array(np.transpose(np.vstack([left_x_predictions, y_points])))
    right_line_window = np.array(np.flipud(np.transpose(np.vstack([right_x_predictions, y_points]))))
    
    poly_points = np.vstack([left_line_window, right_line_window])
    
    cv2.fillPoly(out_img, np.int_([poly_points]), [0,255, 0])
    
    unwarped = cv2.warpPerspective(out_img, Minv, img_size , flags=cv2.INTER_LINEAR)

    result = cv2.addWeighted(img, 1, unwarped, 0.3, 0)
    
    # compute the radius of curvature
    num_of_rows = warped.shape[0]
    left_curvature = simple_curvature(left_x_predictions, num_rows)
    right_curvature = simple_curvature(right_x_predictions, num_rows)
    average_curvature = (left_curvature + right_curvature)/2
    curvature_for_print = "Radius of curvature: %.2f m" % average_curvature
    cv2.putText(result,curvature_for_print , (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,0,0), thickness=2)
    
    #compute the offset from the center
    lane_center = (right_x_predictions[num_rows-1] + left_x_predictions[num_rows-1])/2
    xm_per_pix = 3.7/700 # # US regulation lane width = 3.7m meters 
    center_offset_pixels = abs(img_size[0]/2 - lane_center)
    center_offset_mtrs = xm_per_pix*center_offset_pixels
    distance_from_center = "Distance from center: %.2f m" % center_offset_mtrs
    cv2.putText(result, distance_from_center, (100, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,0,0), thickness=2)
    
    return result

## Test our pipeline on a single image

In [ ]:
import matplotlib.image as mpimg

img = mpimg.imread('test_images/test2.jpg')

# Reinitialize some global variables.
polyfit_left = None
polyfit_right = None
past_good_right_lines = []
past_good_left_lines = []
running_mean_difference_btn_lines = 0


processed = pipeline(img)

# Plot the 2 images
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(processed, cmap='gray')
ax2.set_title('Processed Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
from moviepy.editor import VideoFileClip

# Reinitialize some global variables.
polyfit_left = None
polyfit_right = None
past_good_right_lines = []
past_good_left_lines = []
running_mean_difference_btn_lines = 0

output = 'project_video_output.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(pipeline) 
%time white_clip.write_videofile(output, audio=False)

In [39]:
from moviepy.editor import VideoFileClip

# Reinitialize some global variables.
polyfit_left = None
polyfit_right = None
past_good_right_lines = []
past_good_left_lines = []
running_mean_difference_btn_lines = 0

output = 'challenge_video_output.mp4'
clip1 = VideoFileClip("challenge_video.mp4")
white_clip = clip1.fl_image(pipeline) 
%time white_clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video challenge_video_output.mp4
[MoviePy] Writing video challenge_video_output.mp4


100%|████████████████████████████████████████| 485/485 [03:25<00:00,  2.49it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge_video_output.mp4 

Wall time: 3min 29s


In [40]:
from moviepy.editor import VideoFileClip

# Reinitialize some global variables.
polyfit_left = None
polyfit_right = None
past_good_right_lines = []
past_good_left_lines = []
running_mean_difference_btn_lines = 0

output = 'harder_challenge_video_output.mp4'
clip1 = VideoFileClip("harder_challenge_video.mp4")
white_clip = clip1.fl_image(pipeline) 
%time white_clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video harder_challenge_video_output.mp4
[MoviePy] Writing video harder_challenge_video_output.mp4


100%|█████████████████████████████████████▉| 1199/1200 [09:30<00:00,  2.34it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: harder_challenge_video_output.mp4 

Wall time: 9min 34s
